In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

┌ Info: Precompiling Rocket [df971d30-c9d6-4b37-b8ff-e965b2cb3a40]
└ @ Base loading.jl:1278
┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.
┌ Info: Precompiling GraphPPL [b3f8163a-e979-4e85-b43e-1f63d8c8b42c]
└ @ Base loading.jl:1278
┌ Warning: Package GraphPPL does not have ReactiveMP in its dependencies:
│ - If you have GraphPPL checked out for development and have
│   added ReactiveMP as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to repor

In [2]:
@model function smoothing(n_observations, noise_mean, noise_var)
    
    x_prior_mean = datavar(Float64)
    x_prior_var  = datavar(Float64)
    
    x_prior ~ NormalMeanVariance(x_prior_mean, x_prior_var)

    x = randomvar(n_observations)
    y = datavar(Float64, n_observations)

    x[1] = x_prior

    for i in 2:n_observations
        x[i] ~ x[i - 1] + 1.0
        y[i] ~ x[i] + NormalMeanVariance(noise_mean, noise_var)
    end

    return x_prior_mean, x_prior_var, x, y
end

smoothing (generic function with 1 method)

In [3]:
noise_real_mean = -10.0
noise_real_var  = 100.0

n = 500
data = map(d -> d + rand(Normal(noise_real_mean, sqrt(noise_real_var))), collect(1:n));

In [4]:
function inference(; data, x_prior)
    n = length(data)
    
    _, (x_prior_mean, x_prior_var, x, y) = smoothing(n, noise_real_mean, noise_real_var);

    subscriptions = Vector{Teardown}(undef, n)
    marginals = Vector{Marginal}(undef, n)
    
    for i in 1:n
         @inbounds subscriptions[i] = subscribe!(getmarginal(x[i]), (m) -> marginals[i] = m)
    end
    
    update!(x_prior_mean, mean(x_prior))
    update!(x_prior_var, var(x_prior))
    
    for i in 1:n
        @inbounds update!(y[i], data[i])
    end
    
    foreach(unsubscribe!, subscriptions)
    
    return marginals
end

inference (generic function with 1 method)

In [5]:
inference(
    data = data,
    x_prior = NormalMeanVariance(0.0, 10000.0)
)

┌ Warning: Unused data variable has been found: y_1
└ @ ReactiveMP /Users/bvdmitri/.julia/dev/ReactiveMP/src/model.jl:62


500-element Array{Marginal,1}:
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=0.8804935678108877, v=0.20039678563555818))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=1.8804935678108876, v=0.20039678563555818))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=2.880493567810887, v=0.20039678563555818))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=3.880493567810887, v=0.20039678563555816))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=4.880493567810888, v=0.2003967856355582))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=5.880493567810888, v=0.20039678563555818))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=6.88049356781089, v=0.2003967856355582))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=7.8804935678108885, v=0.20039678563555818))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=8.88